In [1]:
from pathlib import Path

import numpy as np
import pandas as pd
import SimpleITK as sitk

from src.evaluation.scores import dice
from src.resampling.utils import get_np_volume_from_sitk
from src.resampling.resampling import resample_np_binary_volume

In [10]:
# If you use NiftyNet the output must be renamed to match the one requested for the submission
!python src/niftynet/rename_output.py data/segmentation_output/ data/segmentation_output_renamed

In [18]:
prediction_folder = 'data/segmentation_output_renamed'
groundtruth_folder = 'data/hecktor_nii/'
bb_filepath = 'data/bbox.csv'

In [21]:
# List of the files in the validation
prediction_files = [
            f
            for f in Path(prediction_folder).rglob('*.nii.gz')
        ]

# The list is sorted, so it will match the list of ground truth files
prediction_files.sort(key=lambda x: x.name.split('_')[0])   
print(len(prediction_files))
# List of the patient_id in the validation
patient_name_predictions = [
    f.name.split('.')[0] for f in prediction_files
]


21


In [22]:
# List of the ground truth files
groundtruth_files = [
    f for f in Path(groundtruth_folder).rglob('*gtvt.nii.gz') if f.name.split('_')[0] in patient_name_predictions
]
# The list is sorted to match the validation list
groundtruth_files.sort(key=lambda x: x.name.split('_')[0])

print(len(groundtruth_files))

21


In [23]:
# The bounding boxes will be used to compute the Dice score within.
bb_df = pd.read_csv(bb_filepath).set_index('PatientID')

In [24]:
# DataFrame to store the results
results_df = pd.DataFrame(columns=['PatientID', 'Dice Score'])

for i, f in enumerate(prediction_files):
    print(i)
    gt_file = groundtruth_files[i]
    patient_name = f.name.split('.')[0]
    print('Evaluating patient {}'.format(patient_name))
    bb = (bb_df.loc[patient_name, 'x1'], bb_df.loc[patient_name, 'y1'], bb_df.loc[patient_name, 'z1'],
          bb_df.loc[patient_name, 'x2'], bb_df.loc[patient_name, 'y2'], bb_df.loc[patient_name, 'z2'])
    
    sitk_pred = sitk.ReadImage(str(f.resolve()))
    sitk_gt = sitk.ReadImage(str(gt_file.resolve()))
    # Transform from SimpleITK to numpy, otherwise the bounding boxes axis are swapped
    np_pred, px_spacing_pred, origin_pred = get_np_volume_from_sitk(sitk_pred)
    np_gt, px_spacing_gt, origin_gt = get_np_volume_from_sitk(sitk_gt)
    
    # Resample back to the original resolution and crop in the bounding box
    np_pred = resample_np_binary_volume(np_pred, origin_pred, px_spacing_pred, px_spacing_gt, bb)
    np_gt = resample_np_binary_volume(np_gt, origin_gt, px_spacing_gt, px_spacing_gt, bb)

    # Store the results
    results_df = results_df.append({'PatientID': patient_name, 'Dice Score': dice(np_gt, np_pred), 
                                    }, ignore_index=True)

0
Evaluating patient CHGJ007
1
Evaluating patient CHGJ015
2
Evaluating patient CHGJ034
3
Evaluating patient CHGJ055
4
Evaluating patient CHMR013
5
Evaluating patient CHUM007
6
Evaluating patient CHUM021
7
Evaluating patient CHUM042
8
Evaluating patient CHUM043
9
Evaluating patient CHUM044
10
Evaluating patient CHUM049
11
Evaluating patient CHUM056
12
Evaluating patient CHUM065
13
Evaluating patient CHUS007
14
Evaluating patient CHUS021
15
Evaluating patient CHUS031
16
Evaluating patient CHUS039
17
Evaluating patient CHUS046
18
Evaluating patient CHUS057
19
Evaluating patient CHUS064
20
Evaluating patient CHUS097


In [25]:
results_df

,PatientID,Dice Score
0,CHGJ007,0.895127
1,CHGJ015,0.684128
2,CHGJ034,0.769137
3,CHGJ055,0.562072
4,CHMR013,0.315679
5,CHUM007,0.819990
6,CHUM021,0.899573
7,CHUM042,0.907067
8,CHUM043,0.000000
9,CHUM044,0.918355
